Let's inspect the file `sd_xl_base_1.0_0.9vae.safetensors`

In [1]:
import torch
from safetensors import safe_open

In [2]:
file = '../../../../.hf-cache/stable-diffusion-xl-base-1.0/sd_xl_base_1.0_0.9vae.safetensors'

In [3]:
tensors = {}
with safe_open(file, framework='pt', device='cpu') as f:
   for key in f.keys():
       tensors[key] = f.get_tensor(key)

In [4]:
from util import print_as_nested_dict

In [5]:
print_as_nested_dict(tensors.keys(), lv=2)

conditioner
        embedders
first_stage_model
        decoder
        encoder
        post_quant_conv
        quant_conv
model
        diffusion_model


In [9]:
print_as_nested_dict(tensors.keys(), contains='model.diffusion_model', lv=3)

model
        diffusion_model
                input_blocks
                label_emb
                middle_block
                out
                output_blocks
                time_embed


In [11]:
print_as_nested_dict(tensors.keys(), contains='first_stage_model', lv=3)

first_stage_model
        decoder
                conv_in
                conv_out
                mid
                norm_out
                up
        encoder
                conv_in
                conv_out
                down
                mid
                norm_out
        post_quant_conv
                bias
                weight
        quant_conv
                bias
                weight


___

In [7]:
import pickle

In [8]:
with open('state_dict_mapping.pkl', 'rb') as f:
    state_dict_mapping = pickle.load(f)

In [9]:
from diffusers import StableDiffusionXLPipeline

In [10]:
unet = StableDiffusionXLPipeline.from_pretrained('weights/sdxlvae').unet

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [44]:
modified_statedict = diffeng.model.diffusion_model.state_dict()

In [74]:
for k,v in unet.state_dict().items():
    assert v.shape==diffeng.model.diffusion_model.state_dict()[state_dict_mapping[k]].shape
    modified_statedict[state_dict_mapping[k]] = v

In [75]:
diffeng.model.diffusion_model.load_state_dict(modified_statedict)

<All keys matched successfully>

In [53]:
import torch

weight_in_diffusers = unet.state_dict()[k]
weight_in_controlnet= diffeng.model.diffusion_model.state_dict()[state_dict_mapping[k]]

# as sanity check, see if one entry is equal
assert weight_in_diffusers.shape==weight_in_controlnet.shape
assert torch.all((weight_in_diffusers==weight_in_controlnet).bool())

Now, the controlnet-xs model should be loaded correctly, so let's use it!

In [61]:
diffeng = diffeng.to('cuda')